In [19]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../')

from sqlalchemy import create_engine, text

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

USER_DB = POSTGRES_UTEA['USER']
PASS_DB = POSTGRES_UTEA['PASSWORD']
HOST_DB = POSTGRES_UTEA['HOST']
PORT_DB = POSTGRES_UTEA['PORT']
NAME_DB = POSTGRES_UTEA['DATABASE']

ENGINE = create_engine(f'postgresql+psycopg://{USER_DB}:{PASS_DB}@{HOST_DB}:{PORT_DB}/{NAME_DB}')

In [22]:
#path_xlsx_avance = r'G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/AVANCE DE COSECHA V1.xlsx'
path_xlsx_avance = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [23]:
xlsx_resumen = pd.read_excel(path_xlsx_avance, sheet_name='AVANCE_FECHAS')

In [24]:
# elimina el ultimo registro, para descartar los totales
xlsx_resumen = xlsx_resumen.drop(xlsx_resumen.index[-1])

In [25]:
xlsx_resumen

COD_COS                FECHA       AREA    ESTIMADO  ENTREGADO
0          1  2025-06-05 00:00:00  12.279029  875.812683     815.18
1          1  2025-06-10 00:00:00        NaN         NaN    1958.91
2          1  2025-06-13 00:00:00        NaN         NaN     774.88
3          1  2025-06-15 00:00:00        NaN         NaN     574.18
4          1  2025-06-16 00:00:00        NaN         NaN     341.67
...      ...                  ...        ...         ...        ...
7007     634  2025-08-19 00:00:00        NaN         NaN      77.93
7008     634  2025-08-27 00:00:00        NaN         NaN      66.02
7009     634  2025-08-29 00:00:00        NaN         NaN      84.01
7010     634  2025-09-03 00:00:00        NaN         NaN      89.83
7011     634          (en blanco)        NaN         NaN      35.14

[7012 rows x 5 columns]

In [26]:
def colapsar_tabla(df_param):
    suma_acumulador = [None] * len(df_param)
    acumulador = 0
    contador = 0
    for i, r in df_param.iterrows():
        if(r['AREA'] == 0):
            acumulador += r['ENTREGADO']
            contador += 1
        else:
            acumulador += r['ENTREGADO']
            suma_acumulador[contador] = acumulador
            acumulador = 0
            contador += 1
            continue
    if suma_acumulador[-1] == None:
        suma_acumulador[-1] = acumulador

    df_param['ENTREGAS'] = suma_acumulador
    
    df_param = df_param.dropna(subset=['ENTREGAS'])
    df_param = df_param.drop(columns=['ENTREGADO'])
    #df_param = df_param[df_param['AREA']!=0]
    return df_param

In [27]:
list_cod_cos = list(set(xlsx_resumen['COD_COS']))

In [28]:
list_df = []

In [29]:
for i in list_cod_cos:
    grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
    result = colapsar_tabla(grupo)
    list_df.append(result)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_54420\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_54420\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  grupo = xlsx_resumen[xlsx_resumen['COD_COS']==i].fillna(0)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_54420\2327553473.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

In [30]:
df_combinado = pd.concat(list_df, ignore_index=True)

In [31]:
df_combinado

COD_COS                FECHA       AREA     ESTIMADO  ENTREGAS
0           1  2025-06-05 00:00:00  12.279029   875.812683    815.18
1           1  2025-06-18 00:00:00  65.491810  3654.051272   4299.72
2           1  2025-06-28 00:00:00  49.087510  3178.305693   3824.55
3           1  2025-07-02 00:00:00  18.103557  1223.307570   1095.67
4           1  2025-07-20 00:00:00   4.239158   244.377757    814.86
...       ...                  ...        ...          ...       ...
4037      633  2025-08-11 00:00:00   1.985442   110.964344     38.62
4038      633  2025-08-14 00:00:00   1.200234    67.079840     64.11
4039      633  2025-08-29 00:00:00   2.192380   122.529903    126.45
4040      633          (en blanco)   0.000000     0.000000    113.23
4041      634          (en blanco)   0.000000     0.000000   1197.42

[4042 rows x 5 columns]

In [32]:
df_combinado.to_excel(r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')
#df_combinado.to_excel('G:/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - AVANCE COSECHA/2024/AVANCE_SEMANAL/_TEMP_AVANCE_FECHAS.xlsx', index=False, engine='openpyxl', sheet_name='data')

In [33]:
df_combinado.head(2)

COD_COS                FECHA       AREA     ESTIMADO  ENTREGAS
0        1  2025-06-05 00:00:00  12.279029   875.812683    815.18
1        1  2025-06-18 00:00:00  65.491810  3654.051272   4299.72

In [34]:
def cargar_avance_fechas_a_db(df):
    df = df.rename(columns={
            'COD_COS': 'cod_cos',
            'FECHA': 'fecha',
            'AREA': 'area',
            'ESTIMADO': 'estimado',
            'ENTREGAS': 'entregas'
    })
    
    df['cod_cos']     = df['cod_cos'].astype('Int64')
    #df['fecha']   = pd.to_datetime(df["fecha"])
    df['area']  = df['area'].astype('float')
    df['estimado']   = df['estimado'].astype('float')
    df['entregas']   = df['entregas'].astype('float')

    #validad divicion entre 0
    df["tch_estimado"] = np.where(df["area"] > 0,
                              df["estimado"] / df["area"],
                              0)
    #validad divicion entre 0
    df["tch_entregas"] = np.where(df["area"] > 0,
                              df["entregas"] / df["area"],
                              0)
    
    df['diferencia_tn'] = df['entregas'] - df['estimado']
    df["porcen_dif"] = np.where(df["entregas"] > 0,
                              df["diferencia_tn"] / df["entregas"],
                              0)

    with ENGINE.begin() as conn:  # Inicia transacción
        # Vaciar la tabla y reiniciar secuencia
        conn.execute(text(f'TRUNCATE TABLE catastro_iag.data_avance_fechas RESTART IDENTITY'))
        
        # Insertar nuevos datos
        df.to_sql(
            name='data_avance_fechas',
            con=conn,  # conexión cruda dentro de la transacción
            schema='catastro_iag',
            if_exists='append',
            index=False,
            method='multi',
            chunksize=1000
        )
    print(f"✅ Se ha actualziado la tabla de codigos cosecha")

In [35]:
cargar_avance_fechas_a_db(df_combinado)

✅ Se ha actualziado la tabla de codigos cosecha
